In [1]:
import numpy as np
import sympy as sy

In [2]:
x, y, z, k = sy.symbols('x y z k', commutative=False)
A = sy.Function('A')(x)
Psi = sy.Function('Psi')(x)

expr = k*A*k*Psi
expr2 = k*A*k*Psi + k*Psi

In [9]:
def discretize(expr):
    expr = sy.expand(expr)
    
    if expr.func == sy.Add:
        terms = []
        for term in expr.args:
            terms.append(discretize(term))
        res = sy.Add(*tuple(terms))
    else:
        # First find a k (in terms of an index for the expr.args list)
        index = 'None'
        for i in range(len(expr.args)):
            if expr.args[i] == k:
                index = i
    
        # Only change anything if there was at least one k in the expression.
        if not index == 'None':
            #Rafal wants three terms: left side, right side, derivative operators!
            # Take everything behind the k
            temp = sy.Mul(*expr.args[index+1:])
            # apply the derivative operator
            temp = (temp.subs(x, x+1) - temp.subs(x, x-1))/2
            # multiply by whatever was in front of the derivative operator
            res = sy.Mul(sy.Mul(*expr.args[:index]), temp)
            # expand to get standard form
            res = sy.expand(res)
            # in order to check whether there are derivatives left, run on the discretizer on the result again
            res = discretize(res)
        else:
            res = expr
        
    return res

In [14]:
expr3 = (k**2+k)*Psi

In [15]:
expr3 = sy.expand(expr3)

In [16]:
sy.srepr(expr3)

"Add(Mul(Symbol('k'), Function('Psi')(Symbol('x'))), Mul(Pow(Symbol('k'), Integer(2)), Function('Psi')(Symbol('x'))))"